<a href="https://colab.research.google.com/github/rossellacampanale/Tesi_APE/blob/main/Comet_Score_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comet Score Tutorial

This tutorial doesn't come with any sample data files.
Upload your own from the Files tab on the left.

In [ ]:
pip install unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 31.6 MB/s eta 0:00:00


Let's load a model to compute the scores.
A full list of available models can be found here: https://github.com/Unbabel/COMET/blob/master/MODELS.md

In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Let's load data from files into an array of src, mt, ref sentences:
This example assumes the file names srcs.txt for source data,
hyps.txt for the MT output to score,
refs.txt for the reference translations.
Adapt the code to your own file names.

In [ ]:
with open("srcs.txt") as f:
    srcs = [line.strip() for line in f]

with open("hyps.txt") as f:
    hyps = [line.strip() for line in f]

with open("refs.txt") as f:
    refs = [line.strip() for line in f]

data = []
for idx, line in enumerate(srcs):
    data.append({"src": srcs[idx], "mt": hyps[idx], "ref": refs[idx]})



Let's look at the data:

In [ ]:
print(data[4])

{'src': "La Commissione europea ha inoltre continuato a promuovere l'allineamento dei meccanismi e delle pratica di controllo a livello nazionale.", 'mt': "La Commissione europea ha, inoltre, continuato a promuovere l'allineamento dei meccanismi e delle pratiche di controllo nazionali.", 'ref': "La Commissione europea ha inoltre continuato a promuovere l'allineamento dei meccanismi e delle pratiche di controllo a livello nazionale."}


Now we compute the score.


In [ ]:
model_output = model.predict(data, batch_size=8, gpus=1)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  5.40it/s]


let's look at the results: we show the score for each individual sentence, and the average or system score.

In [ ]:
print(model_output['scores'])

print(f"System score: {model_output['system_score']:.3f}")


[0.9208365678787231, 0.9163798689842224, 0.9209128022193909, 0.9083756804466248, 0.9641139507293701, 0.9552477598190308, 0.9076361656188965, 0.9532154202461243, 0.9149537682533264, 0.9528688788414001]
System score: 0.931
